In [5]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('pose_classification_model.h5')


In [2]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import mediapipe as mp

# Load the saved model and recompile
model = load_model('pose_classification_model.h5')
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('./ggg.mp4')

# Initialize variables for tracking state
prev_class = None
prev_class1 = None
prev_prob = 0
prev_prob1 = 0
counter = 0
false_counter = 0

window_name = 'Webcam Feed'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 500, 640)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        results = holistic.process(image)
        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw landmarks on the frame
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        try:
            if results.pose_landmarks:
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                
                row = pose_row
                
                # Convert pose landmarks to a DataFrame and make predictions
                X = pd.DataFrame([row])
                predictions = model.predict(X)
                dead_lift_prob = predictions[0][0]  # Assuming model.predict(X) returns a 2D array with shape (1, 1)
                dead_lift_class = 'up' if dead_lift_prob > 0.5 else 'down'
                
                current_prob = dead_lift_prob

                # Debugging logs
                print(f"Predicted Class: {dead_lift_class}")
                print(f"Predicted Probability: {dead_lift_prob:.2f}")
                print(f"Current Probability: {current_prob:.2f}")
                print(f"Previous Class: {prev_class}, Previous Probability: {prev_prob:.2f}")
                
                # Check conditions for incrementing the counters
                if prev_class == 'down' and prev_prob <= 0.25 and dead_lift_class == 'up' and current_prob >= 0.75:
                    counter += 1
                    print(f"Counter incremented to: {counter}")
                    prev_class = None  # Reset the previous class to ensure proper counting
                    prev_prob = 0
                elif prev_class1 == 'up' and prev_prob1 < 0.75 and dead_lift_class == 'down' and current_prob <= 0.25:
                    false_counter += 1
                    print(f"False Counter incremented to: {false_counter}")
                    prev_class1 = None  # Reset the previous class to ensure proper counting
                    prev_prob1 = 0
                elif prev_class1 == 'down' and prev_prob1 > 0.25 and dead_lift_class == 'up' and current_prob >= 0.75:
                    false_counter += 1
                    print(f"False Counter incremented to: {false_counter}")
                    prev_class1 = None  # Reset the previous class to ensure proper counting
                    prev_prob1 = 0
                else:
                    if dead_lift_class == 'up' and current_prob >= 0.75:
                        prev_class = dead_lift_class
                        prev_prob = current_prob
                    if dead_lift_class == 'down' and current_prob <= 0.25:
                        prev_class = dead_lift_class
                        prev_prob = current_prob
                    elif dead_lift_class == 'up' and current_prob < 0.75 and prev_class =='down':
                        prev_class1 = dead_lift_class
                        prev_prob1 = current_prob
                    elif dead_lift_class == 'down' and current_prob > 0.25 and prev_class =='up':
                        prev_class1 = dead_lift_class
                        prev_prob1 = current_prob
                
                # Get coordinates for drawing
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [640, 480]).astype(int))
                
                # Draw rectangles and text on the image
                cv2.putText(image, dead_lift_class, coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                cv2.rectangle(image, (0, 0), (300, 90), (245, 117, 16), -1)
                
                cv2.putText(image, 'CLASS', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, dead_lift_class.split(' ')[0], (90, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(current_prob, 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Display the counters on the image
                cv2.putText(image, f'Counter: {counter}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, f'False Counter: {false_counter}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except Exception as e:
            print(f"Error: {e}")
            pass
                        
        cv2.imshow(window_name, image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


c:\Users\Fer-3oun\Downloads\pro2\.conda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Predicted Class: down
Predicted Probability: 0.21
Current Probability: 0.21
Previous Class: None, Previous Probability: 0.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


c:\Users\Fer-3oun\Downloads\pro2\.conda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Fer-3oun\Downloads\pro2\.conda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Predicted Class: down
Predicted Probability: 0.21
Current Probability: 0.21
Previous Class: down, Previous Probability: 0.21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Class: down
Predicted Probability: 0.16
Current Probability: 0.16
Previous Class: down, Previous Probability: 0.21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Class: down
Predicted Probability: 0.14
Current Probability: 0.14
Previous Class: down, Previous Probability: 0.16


...

c:\Users\Fer-3oun\Downloads\pro2\.conda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Fer-3oun\Downloads\pro2\.conda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Class: down
Predicted Probability: 0.34
Current Probability: 0.34
Previous Class: up, Previous Probability: 0.88
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Class: down
Predicted Probability: 0.32
Current Probability: 0.32
Previous Class: up, Previous Probability: 0.88


c:\Users\Fer-3oun\Downloads\pro2\.conda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Fer-3oun\Downloads\pro2\.conda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Class: down
Predicted Probability: 0.35
Current Probability: 0.35
Previous Class: up, Previous Probability: 0.88
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Class: down
Predicted Probability: 0.35
Current Probability: 0.35
Previous Class: up, Previous Probability: 0.88


c:\Users\Fer-3oun\Downloads\pro2\.conda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
